In [14]:
from dotenv import load_dotenv
import pandas as pd
import os
import base64
import requests

load_dotenv()

True

In [15]:
OUTPUT_FOLDER = "../generated_GUIs/"

IMAGE_INPUT_FOLDER = "../data/s2w_sample1/"

s2w = pd.read_csv("../data/Screen2Words.csv")
s2w = s2w.drop('Unnamed: 0', axis=1)
sample = pd.read_csv("../data/s2w_sample.csv")
sample = sample.drop("Unnamed: 0", axis=1)

In [16]:
def encode_image(id):
    image_path = IMAGE_INPUT_FOLDER + f"{id}.jpg"
    print(image_path)
    with open(image_path,"rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    

In [17]:
def describe_image(id):
    api_key = os.getenv("OPENAI_API_KEY")
    base64_image = encode_image(id)
    
    summaries = s2w[s2w["screenId"]==id]["summary"].values
    summaries_str = "\n".join([f"Summary {i+1}: {summary}" for i, summary in enumerate(summaries)])


    print(base64_image)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "system",
                "content": [
                {
                    "type": "text",
                    "text": "You will be provided with a screenshot of a mobile screen and five short descriptions of this given mobile screen in natural language. Your task is to create a new description, that includes every information from the given descriptions, as well as functionalities within the provided screen, that are missing in the given text descriptions"
                }]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{summaries_str}"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "text",
                        "text": "Your respone shoukd"
                    }
                ]
            },
        ],
        "max_tokens": 150
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response

In [18]:
response = describe_image(33)

../data/s2w_sample1/33.jpg
...


In [19]:
response.json().get("choices")[0].get("message").get("content")

"The displayed mobile screen is the home page of an application themed around the concept of 'Pitch', presenting a grid of interactive buttons with numbers for selection. This interface contains a variety of numbers including multiple instances of '0', '1', and '2' set against a vibrant pink background, offering users the functionality to perhaps select or input a pitch value. At the bottom of the screen, there is also an advertisement for 'Gutter and Downspout Cleaning' and 'full service lawn and landscape care' from Pride-Outdoor.com."